In [ ]:
import numpy as np
from sklearn.neighbors import NearestCentroid
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances, pairwise_kernels
from KKMeans.elkan import init_lbounds, update_elkan, calc_center_movement
from time import time

def visualize(data, labels):
    if len(data[0]) > 3:
        raise Exception("Dimensionality is too high for visualization")
    elif len(data[0]) == 1:
        plt.scatter(data, [0 for x in range(len(data))], c = labels)
    elif len(data[0]) == 2:
        plt.scatter(data[:,0], data[:,1], c = labels)
    elif len(data[0]) == 3:
        fig = plt.figure()
        ax = fig.add_subplot(projection = "3d")
        ax.scatter(data[:,0], data[:,1], data[:,2], c = labels)


In [ ]:
class KKmeans():
    def __init__(self, seed=None, n_c=5, max_iter=200, q_metric=None, tol=0):
        self.seed = seed
        self.n_c = n_c
        self.X = None
        self.kmatrix = None
        self.rng = np.random.default_rng(seed)
        self.max_iter = max_iter
        self.q_metric = q_metric
        self.tol = tol
    
    def rand_labels(self):
        return self.assign_labels(self.rng.choice(self.X, self.n_c))

    
    def assign_labels(self, centers):
        start = time()
        X_center_kmatrix = pairwise_kernels(self.X, centers)
        dists = np.zeros((len(self.X), self.n_c))
        for cluster in range(self.n_c):
            dists[:, cluster] += (-2* X_center_kmatrix[:, cluster]
                                + pairwise_kernels([centers[cluster]])).reshape(len(self.X),)
        print("assign", time() - start)
        return np.asarray(np.argmin(dists, axis=1), dtype=np.int_)


    def init_elkan(self, labels):
        lbounds = init_lbounds(self.kmatrix, labels, self.n_c)
        labels = np.asarray(np.argmin(lbounds, axis=1), dtype=np.int_, order="C")
        ubounds = lbounds[list(range(len(lbounds))), labels]
        return lbounds, ubounds, labels

    def elkan(self, X, labels=None):
        self.kmatrix = pairwise_kernels(X)
        self.X=X
        if labels is None:
            labels = self.rand_labels()
        quality = 0
        for _ in range(self.max_iter):
            if _ == 0:
                labels_old = np.copy(labels)
                lbounds, ubounds, labels = self.init_elkan(labels_old)
            
            
            c_movement = calc_center_movement(self.kmatrix, labels, labels_old, self.n_c)

            labels_old = np.copy(labels)

                
            for c in range(self.n_c):
                lbounds[:, c] -= c_movement[c]

            for x in range(len(self.X)):
                ubounds[x] += c_movement[labels[x]]
            start = time()
            lbounds, ubounds, labels = update_elkan(self.kmatrix, labels, lbounds, ubounds, self.n_c)
            end = time()
            

            quality, converged = self.measure_iter(np.square(lbounds), labels, labels_old, quality)
            print(end -start)
            if converged: break
        return labels

    def reset_rng(self):
        self.rng = np.random.default_rng(self.seed)

    def inertia(self, labels):
        centers = self.calc_centers(labels)
        squared_distances = np.zeros(len(self.X))
        for i in range(len(self.X)):
            squared_distances[i] = np.linalg.norm(self.X[i] - centers[labels[i]])**2

        return squared_distances
    
    def measure_iter(self, sq_distances, labels, labels_old, quality):
        converged = False
        if self.tol != 0:
            quality_old = quality
            quality = self._calc_q_metric(sq_distances, labels)
            if abs(quality - quality_old) <= self.tol:
                converged = True

        if all(labels_old == labels):
            if self.tol == 0:
                quality = self._calc_q_metric(sq_distances, labels)
            converged = True
        
        return quality, converged
    
    def _calc_q_metric(self, sq_distances, labels):
        if self.q_metric == "inertia":
            return sq_distances[range(sq_distances.shape[0]), labels].sum()
        else:
            raise NotImplementedError(str(self.q_metric) + " quality metric not implemented")
        
    def calc_centers(self, labels):
        clf = NearestCentroid()
        clf.fit(self.X, labels)
        return clf.centroids_
    

In [ ]:
labels = [4, 4, 7, 4, 4, 4, 6, 7, 0, 7, 4, 4, 7, 9, 6, 4, 7, 3, 6, 5, 2, 7, 4, 9, 1, 2, 6, 7, 7, 8, 4, 7, 1, 7, 0, 4, 7, 0, 8, 4, 4, 0, 1, 0, 4, 7, 5, 6, 0, 0, 8, 7, 4, 4, 4, 7, 1, 7, 0, 2, 4, 0, 7, 7, 4, 7, 3, 4, 9, 4, 7, 5, 4, 7, 7, 5, 0, 3, 7, 7, 8, 4, 8, 7, 7, 2, 5, 7, 8, 4, 4, 7, 7, 7, 4, 9, 7, 7, 7, 6]
labels = np.asarray(labels)
n_c = 50
size = 1000
r_s = 0
x, labels_, centers = make_blobs(size, centers=n_c, return_centers = True, random_state = r_s, n_features = 2)



In [ ]:
kkm = KKmeans(seed=1,n_c=n_c, max_iter=100, q_metric="inertia", tol=1e-4)
start = time()
lab = kkm.elkan(x)
end = time()
print(end - start)

In [ ]:
visualize(x, lab)

In [ ]:
kkm.inertia(lab).sum()

In [ ]:
n_c = 10
size = 100
r_s = 4
labels = [0, 8, 5, 5, 8, 2, 0, 9, 7, 2, 5, 7, 2, 5, 9, 5, 8, 2, 0, 2, 1, 0, 0, 5, 3, 5, 7, 0, 5, 5, 2, 5, 3, 0, 5, 5, 7, 5, 5, 0, 2, 7, 8, 2, 5, 8, 5, 7, 5, 5, 5, 3, 2, 5, 5, 2, 3, 9, 9, 9, 9, 5, 2, 6, 2, 8, 2, 2, 0, 5, 3, 5, 0, 7, 9, 3, 5, 2, 5, 1, 2, 3, 7, 5, 5, 4, 5, 2, 2, 8, 3, 2, 2, 8, 1, 2, 2, 5, 2, 3]

x, labels_, centers = make_blobs(size, centers=n_c, return_centers = True, random_state = r_s, n_features = 2)

kkm = KKmeans(seed=4, n_c=n_c, max_iter=100, q_metric = "inertia", tol=1e-4)
lab = kkm.elkan(x, labels)

In [ ]:
np.sum(kkm.inertia(lab))